In [1]:
import pandas as pd
import nltk
nltk.download("inaugural")
from nltk.corpus import stopwords


[nltk_data] Downloading package inaugural to
[nltk_data]     /Users/KevQuant/nltk_data...
[nltk_data]   Package inaugural is already up-to-date!


In [2]:
from nltk.corpus import inaugural
inaugural.fileids()

for fileid in inaugural.fileids():
    print(fileid)
    print("char",len(inaugural.raw(fileid)))
    print("words",len(inaugural.words(fileid)))
    print("sents",len(inaugural.sents(fileid)))

1789-Washington.txt
char 8619
words 1538
sents 24
1793-Washington.txt
char 791
words 147
sents 4
1797-Adams.txt
char 13877
words 2585
sents 37
1801-Jefferson.txt
char 10137
words 1935
sents 42
1805-Jefferson.txt
char 12908
words 2384
sents 45
1809-Madison.txt
char 7001
words 1265
sents 21
1813-Madison.txt
char 7157
words 1304
sents 33
1817-Monroe.txt
char 19887
words 3693
sents 122
1821-Monroe.txt
char 26326
words 4909
sents 129
1825-Adams.txt
char 17741
words 3150
sents 74
1829-Jackson.txt
char 6817
words 1208
sents 25
1833-Jackson.txt
char 7058
words 1267
sents 30
1837-VanBuren.txt
char 23417
words 4171
sents 95
1841-Harrison.txt
char 49700
words 9165
sents 210
1845-Polk.txt
char 28716
words 5196
sents 153
1849-Taylor.txt
char 6605
words 1182
sents 22
1853-Pierce.txt
char 20081
words 3657
sents 104
1857-Buchanan.txt
char 16815
words 3098
sents 89
1861-Lincoln.txt
char 21017
words 4005
sents 138
1865-Lincoln.txt
char 3926
words 785
sents 27
1869-Grant.txt
char 6503
words 1239
sents 41

Part-1-1: Divide it into two datasets: preWWI (till 1913) and postWWI (after 1917).

In [3]:
preWWI_lst = [] #Create list to collect preWWI (till 1913) fileids
postWWI_lst =[] #Create list to collect postWWI (after 1917) fileids

for fileid in inaugural.fileids():
    if int(fileid[0:4])<=1913: #Collect the fileids before or on 1913
        preWWI_lst.append(fileid)
    elif int(fileid[0:4]) >1917: #Collect the fileids after on 1917
        postWWI_lst.append(fileid)
    else:
        pass

##Part-1-2: Normalize the text for each dataset by down-casing all words (but no stemming or lemmatization).

In [4]:
pre_words_lst = []
for i in range(len(preWWI_lst)):
    pre_words_lst.append(list(inaugural.words(preWWI_lst[i])))
    
pre_words_lst2 = []
for i in range(len(pre_words_lst)):
    pre_words_lst2 = pre_words_lst2 + pre_words_lst[i]    
pre_words_lst2 = [word.lower() for word in pre_words_lst2]
print(len(pre_words_lst2))

pre_words_lst3 = []
for word in pre_words_lst2:
    if word in stopwords.words('english') or len(word)<=2:
        pass
    else:
        pre_words_lst3.append(word)
print(len(pre_words_lst3))

89718
38423


In [7]:
post_words_lst = []
for i in range(len(postWWI_lst)):
    post_words_lst.append(list(inaugural.words(postWWI_lst[i])))
    
post_words_lst2 = []
for i in range(len(post_words_lst)):
    post_words_lst2 = post_words_lst2 + post_words_lst[i]    
post_words_lst2 = [word.lower() for word in post_words_lst2]
print(len(post_words_lst2))

post_words_lst3 = []
for word in post_words_lst2:
    if word in stopwords.words('english') or len(word)<=2:
        pass
    else:
        post_words_lst3.append(word)
print(len(post_words_lst3))

54361
23063


##Part-1-3: For each dataset, compute the frequency distribution (i.e., probability) of unigrams and bigrams.  Then compute the following and write two output files -- one for unigrams and one for bigrams.

##Unigram

In [ ]:
#Count the frequency of words appearing preWWI dataset into dictionary
fduni = nltk.FreqDist(pre_words_lst3)
#Convert words dictionary into dataframe
pre_df = pd.DataFrame(list(zip(fduni.keys(), fduni.values())), columns =["tokens", "freq"])
#Sort the table by frequency values
pre_df = pre_df.sort_values(by=["freq"], ascending=False)
#Calculate the probabilities of the words appearing in the preWWI dataset and include the column in the table
pre_df["pre_prob"] = [round(freq / len(pre_words_lst2),7) for freq in pre_df.freq]
#Sort the table by probailities value
pre_df = pre_df.sort_values(by = ["pre_prob"], ascending = False)
#pre_df = pre_df.iloc[0:355]
#for i in range(0, pre_df.shape[0]):
 #   print("%d%15s %0.7f"%(i,pre_df.iloc[i]["tokens"], pre_df.iloc[i]["pre_prob"]))

In [ ]:
#Count the frequency of words appearing postWWI dataset into dictionary
fduni = nltk.FreqDist(post_words_lst3)
#Convert words dictionary into dataframe
post_df = pd.DataFrame(list(zip(fduni.keys(), fduni.values())), columns =["tokens", "freq"])
#Sort the table by frequency values
post_df = post_df.sort_values(by=["freq"], ascending=False)
#Calculate the probabilities of the words appearing in the preWWI dataset and include the column in the table
post_df["post_prob"] = [round(freq / len(post_words_lst2),7) for freq in post_df.freq]
#Sort the table by probailities value
post_df = post_df.sort_values(by = ["post_prob"], ascending = False)

#post_df = post_df.iloc[0:355]
#post_df
#for i in range(0, post_df.shape[0]):
 #   print("%d%15s %0.7f"%(i,post_df.iloc[i]["tokens"], post_df.iloc[i]["post_prob"]))

In [23]:
#Merge the PreWWI and PostWWI dataframes (already sorted by probabilities in decending order) 
#in "outer" format which means the values show nan if NO tokens matches when merging the table
pre_post_df = pd.merge(pre_df, post_df, on="tokens", how="outer")
#Drop all the nan rows after merging tables
pre_post_df.dropna(axis=0, how="any", inplace= True)
#Extract the top 200 Frequency rows of the table and sort the table by the column "tokens"
pre_post_df = pre_post_df[0:200].sort_values(by =["tokens"])
#Drop the Frequency columns
pre_post_df.drop(columns=["freq_x", "freq_y"], inplace=True)
pre_post_df
#pre_post_df.to_csv("unigrams.txt", index=False,sep="\t")

,tokens,pre_prob,post_prob
176,action,0.000357,0.000497
48,administration,0.000803,0.000331
191,affairs,0.000334,0.000166
166,aid,0.000368,0.000110
161,also,0.000379,0.000349
125,always,0.000457,0.000607
51,american,0.000780,0.001398
50,among,0.000791,0.000662
194,another,0.000334,0.000460
111,authority,0.000490,0.000184


##bigram

In [7]:
pre_bigrams = list(nltk.bigrams(pre_words_lst2))
fdbi = nltk.FreqDist(pre_bigrams)
pre_df = pd.DataFrame(list(zip(fdbi.keys(), fdbi.values())), columns =["tokens", "freq"])
pre_df = pre_df.sort_values(by=["freq"], ascending=False)

pre_df["pre_prob"] = [round(freq / len(pre_words_lst2),7) for freq in pre_df.freq]


pre_df = pre_df.sort_values(by = ["pre_prob"], ascending = False)
#pre_df = pre_df.iloc[0:30000]
#for i in range(0, pre_df.shape[0]):
 #   print("%15s %0.7f"%(pre_df.iloc[i]["tokens"], pre_df.iloc[i]["pre_prob"]))
#stopwords.words('english')

In [13]:
type(pre_bigrams)

list

In [8]:
post_bigrams = list(nltk.bigrams(post_words_lst2))
fdbi = nltk.FreqDist(post_bigrams)
post_df = pd.DataFrame(list(zip(fdbi.keys(), fdbi.values())), columns =["tokens", "freq"])
post_df = post_df.sort_values(by=["freq"], ascending=False)

post_df["post_prob"] = [round(freq / len(post_words_lst2),7) for freq in post_df.freq]


post_df = post_df.sort_values(by = ["post_prob"], ascending = False)
#post_df = post_df.iloc[0:30000]
#for i in range(0, post_df.shape[0]):
 #   print("%15s %0.7f"%(post_df.iloc[i]["tokens"], post_df.iloc[i]["post_prob"]))
#stopwords.words('english')

In [ ]:
pre_words_lst3 = []
for word in pre_words_lst2:
    if word in stopwords.words('english') or len(word)<=2:
        pass
    else:
        pre_words_lst3.append(word)
print(len(pre_words_lst3))

In [9]:
pre_post_df = pd.merge(pre_df, post_df, on="tokens")
pre_post_df = pre_post_df.sort_values(by=["tokens"])
pre_post_df.drop(columns=["freq_x", "freq_y"], inplace=True)
pre_post_df[0:10]
#pre_post_df.to_csv("unigrams.txt", index=False,sep="\t")

,tokens,pre_prob,post_prob
2067,"("", and)",0.000045,0.000074
4325,"("", i)",0.000022,0.000018
2274,"("", in)",0.000045,0.000037
5439,"("", is)",0.000011,0.000018
7771,"("", let)",0.000011,0.000018
5679,"("", our)",0.000011,0.000018
6005,"("", that)",0.000011,0.000018
690,"("", the)",0.000123,0.000055
2722,"("", their)",0.000033,0.000018
694,"("", to)",0.000123,0.000018


In [ ]:
#pre_df.set_index("tokens").join(post_df.set_index("tokens"),)
pre_post_df = pd.merge(pre_df, post_df, on="tokens")
pre_post_df.columns = ["unigram", "pre-WWI","post-WWI" ]
pre_post_df = pre_post_df.sort_values(by = ["pre-WWI"], ascending=False)
pre_post_df = pre_post_df[0:200]
pre_post_df.sort_values(by = ["unigram"])

In [40]:
words_lst3[1]

'-'

In [13]:
#Iterating the preWWI list to down-casing all words
for i in range(len(preWWI_lst)):
    preWWI_lst[i] = inaugural.raw(preWWI_lst[i]).lower() #Store the preWWI data (ALL words) to the same list preWWI_lst

#Iterating the postWWI_lst list to down-casing all words
for i in range(len(postWWI_lst)):
    postWWI_lst[i] = inaugural.raw(postWWI_lst[i]).lower() #Store the postWWI data (ALL words) to the same list postWWI_lst


##Part-1-3: For each dataset, compute the frequency distribution (i.e., probability) of unigrams and bigrams.  Then compute the following and write two output files -- one for unigrams and one for bigrams.

##Unigram

In [32]:
import nltk
from nltk import word_tokenize, sent_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/KevQuant/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
words_lst=[]
for i in range(len(preWWI_lst)):
    words_lst.append( word_tokenize(preWWI_lst[i]))

words_lst2=[]
for i in range(len(words_lst)):
    words_lst2 +=words_lst[i]

words_lst3=[]
for word in words_lst2:
    if (len(word)<=2) or (word in stopwords.words()):
        pass
    else:
        words_lst3.append(word)

fduni = nltk.FreqDist(words_lst3)
len(fduni)

NameError: name 'stopwords' is not defined

In [66]:
len(preWWI_lst)

32

In [67]:
words_dict = {}
for words in preWWI_lst:
    words = word_tokenize(words)
    for word in words:
        if word in stopwords.words() or len(word)<=2:
            pass
        else:
            if word.strip() in words_dict.keys():
                words_dict[word.strip()]+=1
            else:
                words_dict[word.strip()]=1

    

6769

In [73]:
words_dict2 = {}
for k,v in words_dict.items():
    if v >= 200:
        pass
    else:
        words_dict2[k] = v
words_dict2 

6761

In [ ]:
df = pd.DataFrame(list(zip(words_dict2.keys(), words_dict2.values())), columns =["tokens", "freq"])
df.sort_values(by=["tokens"], ascending=True)

In [107]:
preWWI_words=[]
for i in range(len(preWWI_lst)):
    preWWI_words = preWWI_words +preWWI_lst[i]
words = word_tokenize(preWWI_words)
freq_uni = nltk.FreqDist(words)
df = pd.DataFrame(list(zip(freq_uni.keys(), freq_uni.values())), columns = ["tokens", "freq"])
#df.drop(labels=[5])
df.sort_values(by = ["tokens"])
#df.drop(labels=[5])
#df

TypeError: can only concatenate list (not "str") to list

In [111]:
preWWI_words = []
for i in range(len(preWWI_lst)):
    preWWI_words = preWWI_words  + [preWWI_lst[i]]

len(preWWI_words)

32

In [98]:
cnt =0
for words in preWWI_lst:
    cnt += len(words)
cnt

488573

In [117]:
aa = [["abced"], ["fuahf fiaha"]]

aa= aa[0]+aa[1]
word_tokenize(aa[0])

['abced']

In [132]:
words_lst=[]
for i in range(len(preWWI_lst)):
    words_lst.append( word_tokenize(preWWI_lst[i]))

words_lst2=[]
for i in range(len(words_lst)):
    words_lst2 +=words_lst[i]

fduni = nltk.FreqDist(words_lst2)
len(fduni)

6936

In [130]:
cnt=1
for words in words_lst:
    cnt += len(words)
cnt

89345

In [44]:
unigrams = inaugural.words()
print("unigrams:", len(unigrams))
bigrams = list(nltk.bigrams(unigrams))
print("bigram",len(bigrams))

unigrams: 145735
bigram 145734


In [45]:
unigrams

['Fellow', '-', 'Citizens', 'of', 'the', 'Senate', ...]

In [ ]:
fduni = nltk.FreqDist(unigrams)
print("length of unigrams Dict",len(fduni))

for (w,c) in fduni.items():
    print(w,c)

##Check out the frequency of each word link to another word next to it.

In [26]:
cfd =  nltk.ConditionalFreqDist(bigrams)
print(cfd["Fellow"].items())

dict_items([('-', 7), ('citizens', 16), ('Citizens', 1)])


## Get the Conditional Probabilities of 
##(freq("-") + freq("Fellow")) divided by (Total freq("fellow"))

In [27]:
cfd["Fellow"].freq("-")

0.2916666666666667